# 数据操作——代码实现

In [1]:
import torch

**张量（tensor）**：一组数，可以类比numpy中的ndarray（n-dimension array），但是ndarray只能在CPU上运行，tensor可以在GPU上运行

注：不要纠结数学上的张量概念

## 1 张量基本使用方法

In [2]:
x = torch.arange(12) # 创建一个大小为12的一维张量
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [3]:
x.shape # shape 属性可以访问张量的形状

torch.Size([12])

In [4]:
x.numel() # 通过numel，number of elements 函数访问张量的元素总数

12

In [6]:
# 重塑张量：
y = x.reshape(3,4)
y

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [7]:
# 特殊数字的张量
x = torch.zeros(2,3,4)
y = torch.ones(3,2)

print(x)
print(y)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [8]:
# 指定元素创建
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

## 2 张量简单算术运算

In [9]:
# 张量简单运算
x = torch.tensor([1,2,3,4])
y = torch.tensor([5,6,7,8])

x+y, x-y, x*y, x/y, x**y

(tensor([ 6,  8, 10, 12]),
 tensor([-4, -4, -4, -4]),
 tensor([ 5, 12, 21, 32]),
 tensor([0.2000, 0.3333, 0.4286, 0.5000]),
 tensor([    1,    64,  2187, 65536]))

In [10]:
# 计算规则同矩阵运算
x = torch.tensor([1,2,3,4])
y = torch.tensor([5,6,8])
x+y

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [11]:
# 指数运算
z = torch.exp(x)
z

tensor([ 2.7183,  7.3891, 20.0855, 54.5981])

## 3 张量连接运算

In [12]:
x = torch.arange(12, dtype=torch.float32).reshape((3,4))
y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
x,y

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[2., 1., 4., 3.],
         [1., 2., 3., 4.],
         [4., 3., 2., 1.]]))

In [13]:
torch.cat((x,y),dim=0)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 2.,  1.,  4.,  3.],
        [ 1.,  2.,  3.,  4.],
        [ 4.,  3.,  2.,  1.]])

In [14]:
torch.cat((x,y),dim=1)

tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
        [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
        [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]])

## 4 布尔张量

In [15]:
x == y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

## 5 张量求和

In [16]:
x.sum()

tensor(66.)

请注意，对一个张量所有元素进行求和，返回的是一个 1x1 的张量元素

## 6 广播机制

广播机制：broadcasting mechanism，例如 a 是一个 3x1 的矩阵，b 是同维度的 1x2 矩阵，a+b会返回 3x2 矩阵

In [17]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [18]:
a+b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

a,b 是同维矩阵，触发了广播机制， a: 3x1 -> 3x2 ; b: 1x2 -> 3x2 `通过简单复制补足`

所以，现在运算的矩阵是：
a:         
[ [0,0],          
  [1,1],         
  [2,2]
]

b:
[ [0,1],
  [0,1],
  [0,1]
]

## 7 元素访问

In [19]:
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [20]:
x[-1], x[1:3], x[2], x[2,2]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([ 8.,  9., 10., 11.]),
 tensor(10.))

## 8 复制问题与内存

In [21]:
# id: python中的指针
before = id(y)
y = y + x
id(y) == before

False

可以发现，运算后为y重新分配了一份空间，之前的空间已经被收回。

一般而言，这是不好的：
1. 万一有大型张量运算，我们总是更新内存地址，会降低效率
2. 如果有代码的其他部分，引用了本内存空间，会造成程序错误

所以，解决方案如下：

In [22]:
z = torch.zeros_like(y)
z

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [23]:
before = id(z)
z[:] = x+y
id(z) == before

True

In [24]:
# 或者，后续不会用到x
before = id(x)
x += y
id(x) == before

True

## 9 类型转换

In [25]:
# 转为ndarray
a = x.numpy()
b = torch.tensor(a)
type(a), type(b)

(numpy.ndarray, torch.Tensor)

In [26]:
# 大小为1的张量可以转换成python的标量
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)